In [1]:
import numpy as np
import os
import io
import cv2

import torch

import sys
# sys.path.append('/fs/cfar-projects/actionloc/bounce_back/InternVideo/InternVideo2/multi_modality')
sys.path.append('/fs/cfar-projects/inr_analysis/pulkit/InternVideo/InternVideo2/multi_modality')

from config import (Config,
                    eval_dict_leaf)


from utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)

/fs/cfar-projects/inr_analysis/semantic_inr/env2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/fs/cfar-projects/inr_analysis/pulkit/InternVideo/InternVideo2/multi_modality/models/backbones/internvideo2/internvl_clip_vision.py:148: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
DropoutAddRMSNorm of flash_attn is not installed!!!
/fs/cfar-projects/inr_analysis/pulkit/InternVideo/InternVideo2/multi_modality/models/backbones/internvideo2/internvideo2.py:148: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/fs/cfar-projects/inr_analysis/pulkit/InternVideo/InternVideo2/multi_

In [8]:
from importlib import reload
import utils
reload(utils)
from utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)

In [3]:
video = cv2.VideoCapture('example1.mp4')
frames = [x for x in _frame_from_video(video)]

In [4]:
text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
                   "A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.",
                   "A person dressed in a blue jacket shovels the snow-covered pavement outside their house.",
                   "A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner.",
                   "A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.",
                   "A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.",
                   "A playful dog slides down a snowy hill, wagging its tail with delight.",
                   "A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees.",
                   "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
                   "A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery."]

In [6]:
config = Config.from_file('internvideo2_stage2_config.py')
config = eval_dict_leaf(config)

In [7]:
model_pth = '/fs/cfar-projects/inr_analysis/semantic_inr/InternVideo2-stage2_1b-224p-f4.pt'
config['pretrained_path'] = model_pth

In [8]:
intern_model, tokenizer = setup_internvideo2(config)

Using pretrained model: bert-large-uncased


/fs/cfar-projects/inr_analysis/semantic_inr/env2/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/fs/cfar-projects/inr_analysis/pulkit/InternVideo/InternVideo2/multi_modality/models/backbones/internvideo2/internvideo2.py:718: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits 

load_state_dict: _IncompatibleKeys(missing_keys=[], unexpected_keys=['temp', 'itm_head.weight', 'itm_head.bias'])


In [8]:
intern_model

InternVideo2_Stage2(
  (vision_encoder): PretrainInternVideo2(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 1408, kernel_size=(1, 14, 14), stride=(1, 14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0): Block(
        (norm1): RMSNorm()
        (attn): Attention(
          (qkv): Linear(in_features=1408, out_features=4224, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1408, out_features=1408, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
          (q_norm): RMSNorm()
          (k_norm): RMSNorm()
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): RMSNorm()
        (mlp): Mlp(
          (fc1): Linear(in_features=1408, out_features=6144, bias=True)
          (act): GELU(approximate='none')
          (drop1): Dropout(p=0.0, inplace=False)
          (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          (drop2): Dropout(p=0.0, inp

In [9]:
tokenizer

BertTokenizer(name_or_path='bert-large-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
reload(utils)
from utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)

In [10]:
texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=5, config=config)

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')

frames_tensor.shape:  torch.Size([1, 4, 3, 224, 224])
vfeat.shape after encode_vision:  torch.Size([1, 768])
vfeat.shape after vision_proj:  torch.Size([1, 512])
vid_feat.shape:  torch.Size([1, 512])
text_feats_tensor.shape: torch.Size([10, 512])
Predict label: vid_feat.shape: torch.Size([1, 512]), txt_feat.T.shape: torch.Size([512, 10])
text: A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon. ~ prob: 0.7867
text: A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run. ~ prob: 0.0480
text: A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner. ~ prob: 0.0350
text: A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees. ~ prob: 0.0309
text: A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees. ~ prob: 0.0259
